In [1]:
# data comes from http://web.stanford.edu/~ericyi/project_page/part_annotation/index.html
from pyntcloud import PyntCloud

import sys
import os
if sys.platform == 'darwin':
    data_path = os.getcwd() + "/PartAnnotation"
else:
    data_path = os.getcwd() + "\\PartAnnotation"

In [2]:
data_path

'/Users/shijian/git/3D-CNN/3d_pointcloud/PartAnnotation'

In [3]:
def find_data(data_path):
    data = []
    label = []
    for entry in os.scandir(data_path):
        if entry.is_dir():
            for pts_data in os.scandir(os.path.join(data_path, entry.name, 'points')):
                data.append(os.path.join(data_path, entry.name, 'points', pts_data.name))
                label.append(entry.name)
    return data, label

In [4]:
data, label = find_data(data_path)

In [5]:
my_point_cloud = PyntCloud.from_file(data[1000], sep=" ", header=0, names=["x","y","z"])

In [118]:
my_point_cloud.add_structure("voxelgrid", x_y_z=[32, 32, 32])

my_point_cloud.plot(point_size=0.01)

In [6]:
import math
import numpy as np

def voxelize3D(pts, dim=[1,1,1]):
    """
    pts: receives .pts cloud point data. 2D array, arbitary sized X,Y,Z pairs.
    dim: dimensioin of output voxelized data
    
    .pts data X, Y, Z ranged from -0.5~0.5.
    This function will locate the grid cube and calculate the density of each cube.
    """
    assert(pts.shape[1]==3)
    assert(len(dim)==3)
    
    x_grid_length = 1/dim[0]
    y_grid_length = 1/dim[1]
    z_grid_length = 1/dim[2]
    
    output = np.zeros((dim[0],dim[1],dim[2])) # Make a 10 by 20 by 30 array
    
    for pair in pts:
        x_loc = int(pair[0]/x_grid_length) + int(dim[0]/2)
        y_loc = int(pair[1]/y_grid_length) + int(dim[1]/2)
        z_loc = int(pair[2]/z_grid_length) + int(dim[2]/2)
        if output[x_loc, y_loc, z_loc] is None:
            output[x_loc, y_loc, z_loc] = 1
        else:
            output[x_loc, y_loc, z_loc] += 1
            
    return output

In [7]:
vox = voxelize3D(my_point_cloud.xyz, [48,48,48])

In [20]:
# %load ./voxel_grid_plot.py
import os
import shutil

import numpy as np

from IPython.display import IFrame
from matplotlib import pyplot as plt


def plot_voxelgrid(voxelgrid,
                   output_name=None,
                   cmap="Oranges",
                   axis=True,
                   width=800,
                   height=500):

    scaled_shape = voxelgrid.shape

    vector = voxelgrid
    points = np.argwhere(vector) * scaled_shape

    s_m = plt.cm.ScalarMappable(cmap=cmap)
    rgb = s_m.to_rgba(vector.reshape(-1)[vector.reshape(-1) > 0])

    camera_position = points.max(0) + abs(points.max(0))

    look = points.mean(0)

    if axis:
        axis_size = points.ptp() * 1.5
    else:
        axis_size = 0

    placeholders = {}

    placeholders["POINTS_X_PLACEHOLDER"] = points[:, 0].tolist()
    placeholders["POINTS_Y_PLACEHOLDER"] = points[:, 1].tolist()
    placeholders["POINTS_Z_PLACEHOLDER"] = points[:, 2].tolist()

    placeholders["R_PLACEHOLDER"] = rgb[:, 0].tolist()
    placeholders["G_PLACEHOLDER"] = rgb[:, 1].tolist()
    placeholders["B_PLACEHOLDER"] = rgb[:, 2].tolist()

    placeholders["S_x_PLACEHOLDER"] = scaled_shape[0]
    placeholders["S_y_PLACEHOLDER"] = scaled_shape[1]
    placeholders["S_z_PLACEHOLDER"] = scaled_shape[2]

    placeholders["CAMERA_X_PLACEHOLDER"] = camera_position[0]
    placeholders["CAMERA_Y_PLACEHOLDER"] = camera_position[1]
    placeholders["CAMERA_Z_PLACEHOLDER"] = camera_position[2]

    placeholders["LOOK_X_PLACEHOLDER"] = look[0]
    placeholders["LOOK_Y_PLACEHOLDER"] = look[1]
    placeholders["LOOK_Z_PLACEHOLDER"] = look[2]

    placeholders["AXIS_SIZE_PLACEHOLDER"] = axis_size

    placeholders["N_VOXELS_PLACEHOLDER"] = sum(vector.reshape(-1) > 0)

    if output_name is None:
        output_name = "plotVG.html"

    BASE_PATH = os.getcwd()
    src = "{}/{}".format(BASE_PATH, "voxelgrid.html")
    dst = "{}/{}".format(os.getcwd(), output_name)

    with open(src, "r") as inp, open(dst, "w") as out:
        for line in inp:
            for key, val in placeholders.items():
                if key in line:
                    line = line.replace(key, str(val))
            out.write(line)

    return IFrame(output_name, width=width, height=height)

In [21]:
plot_voxelgrid(vox)